# Sistemas Inteligentes

## Practica 1: Búsqueda en espacio de estados

# Estructura

## Clases



### Acción
Clase Acción que representa que es lo que ha de ser hecho, en este caso el origen, el destino y el coste entre estos.




In [55]:
class Accion:
    def __init__(self, origen, destino, coste):
        self.origen = origen
        self.destino = destino
        self.coste = coste

### Estado
Clase estado que contiene el estado del problema, que en este caso es la interseccion donde estamos + el id


In [54]:
class Estado:
    def __init__(self, interseccion):
        self.interseccion = interseccion

### Nodo
Clase que representa un nodo dentro del grafo a resolver. Este contiene un estado, un padre, una accion y su profundidad en el grafo.


In [53]:
class Nodo:
    def __init__(self,  estado, padre=None, accion=None, coste=0, profundidad=0, id=1):
        self.estado = estado
        self.padre = padre
        self.accion = accion
        self.coste = coste
        self.profundidad = profundidad
        self.id = id
    
    def hijo(self, accion):
        estado_resultante = accion  # Suponiendo que las acciones modifican el estado
        return Nodo(estado_resultante, self, accion, self.coste + 1, self.profundidad + 1)
    
    def solucion(self):
        # Reconstruir el camino de la solución desde el nodo inicial
        solucion = []
        nodo_actual = self
        while nodo_actual.padre is not None:
            solucion.append(nodo_actual.accion)
            nodo_actual = nodo_actual.padre
        solucion.reverse()  # Para tener la solución en orden desde el inicio hasta el final
        return solucion

### Problema
Clase que representa el problema a resolver, contiene la informacion del JSON y 


In [52]:
class Problema:
    def __init__(self, ruta_json):
        with open(ruta_json, 'r') as archivo:
            datos_json = json.load(archivo)

        self.distancia = datos_json["distance"]
        self.interseccion = datos_json["intersections"]
        self.estado_inicial = datos_json["initial"]
        self.estado_objetivo = datos_json["final"]
        self.segmentos = datos_json["segments"]
    

    def es_objetivo(self, estado):
        return estado == self.estado_objetivo
    
    def acciones(self, estado):
        # Aquí se deben devolver las acciones posibles desde el estado actual (las intersecciones conectadas)
        return [seg["destination"] for seg in self.segmentos if seg["origin"] == estado]
    
    def resultado(self, estado, accion):
        # El resultado de aplicar una acción es simplemente el destino (intersección) al que se va
        return accion
    
    def coste_individual(self, nodo, accion, s):
        # Calcular el coste de la acción
        for seg in self.segmentos:
            if seg["origin"] == nodo.estado and seg["destination"] == accion:
                return seg["distance"]
        return float('inf')  # Si no se encuentra el segmento

### Busqueda
Clase abstracta que contiene el bucle basico que usan todos los metodos de busqueda,

In [51]:
from abc import ABC,abstractmethod

class Busqueda(ABC):
    def __init__(self, problema):
        self.problema = problema
        self.nodos_explorados = 0
        self.nodos_expandidos = 0
        self.soluciones_generadas = 0
        self.coste = 0
        self.profundidad = 0
        self.frontera = []
        self.cerrados = set()

    def expandir(self, nodo, problema):
        sucesores = []
        for accion in problema.acciones(nodo.estado):
            resultado = problema.resultado(nodo.estado, accion)
            s = Nodo(resultado, nodo, accion)
            s.coste = nodo.coste + problema.coste_individual(nodo, accion, s)
            s.profundidad = nodo.profundidad + 1
            sucesores.append(s)
        return sucesores      
    
    def buscar(self):
        self.insertar_nodo(self.problema.estado_inicial, self.frontera)

        while True:
            # Comprobamos que la frontera no está vacía
            if self.es_vacio(self.frontera):
                # No hay solución
                return None, self.soluciones_generadas, self.nodos_explorados, self.nodos_expandidos, None, None
            
            # Extraemos el primer nodo de la frontera
            nodo = self.extraer_nodo(self.frontera)
            self.nodos_explorados += 1

            # Comprobamos si el nodo es la solución
            if self.problema.es_objetivo(nodo.estado):
                coste = nodo.coste
                profundidad = nodo.profundidad
                return nodo.solucion(), self.soluciones_generadas, self.nodos_explorados, self.nodos_expandidos, coste, profundidad
            
            # Si no es la solución, expandimos los nodos sucesores
            sucesores = self.expandir(nodo)
            self.nodos_expandidos += len(sucesores)
            
            # Concatenar los sucesores a la frontera
            self.concatenar_nodos(self.frontera, sucesores)
            self.soluciones_generadas += len(sucesores)
    
    @abstractmethod
    def insertar_nodo(self, nodo, frontera):
        pass
    @abstractmethod
    def concatenar_nodos(self, frontera, sucesores):
        pass
    @abstractmethod
    def extraer_nodo(self, frontera):
        pass
    @abstractmethod
    def es_vacio(self, frontera):
        pass


### 1: Estrategia de búsqueda no informada

#### 1.1: Búsqueda primero en anchura

In [6]:
def busqueda_en_arbol(problema):
    frontera = []
    nodo_inicial = Nodo(problema.estado_inicial)
    frontera.append(nodo_inicial)

    # Estadísticas
    soluciones_generadas = 0
    nodos_explorados = 0
    nodos_expandidos = 0

    while True:
        # Si la frontera está vacía, devolver FALLO
        if not frontera:
            return None, soluciones_generadas, nodos_explorados, nodos_expandidos, None, None
        
        # Borrar el primer nodo de la frontera
        nodo = frontera.pop(0)
        nodos_explorados += 1
        
        # Si el nodo contiene un estado objetivo, devolver la solución
        if problema.es_objetivo(nodo.estado):
            coste = nodo.coste
            profundidad = nodo.profundidad
            return nodo.solucion(), soluciones_generadas, nodos_explorados, nodos_expandidos, coste, profundidad
        
        # Expandir el nodo y añadir los hijos a la frontera
        sucesores = expandir(nodo, problema)
        nodos_expandidos += len(sucesores)
        frontera.extend(sucesores)

        # Contar las soluciones generadas
        soluciones_generadas += len(sucesores)

def expandir(nodo, problema):
    sucesores = []
    for accion in problema.acciones(nodo.estado):
        resultado = problema.resultado(nodo.estado, accion)
        s = Nodo(resultado, nodo, accion)
        s.coste = nodo.coste + problema.coste_individual(nodo, accion, s)
        s.profundidad = nodo.profundidad + 1
        sucesores.append(s)
    return sucesores

In [66]:
from Classes import *
from Busqueda import Busqueda_Anchura
#Windows
#ruta_json = r'problems\medium\calle_mariÌa_mariÌn_500_0.json'
#MacOs
ruta_json = r'pr1_SSII/problems/medium/calle_mariÌa_mariÌn_500_0.json'

# Crear el objeto problema
problema = Problema(ruta_json)

anchura = Busqueda_Anchura(problema)

# Ejecutar el algoritmo de búsqueda
solucion, soluciones_generadas, nodos_explorados, nodos_expandidos, coste, profundidad = anchura.buscar()

# Mostrar la solución y estadísticas
if solucion:
    print("Solución encontrada:", solucion)
    print("Estadísticas:")
    print(f"Número de soluciones generadas: {soluciones_generadas}")
    print(f"Número de nodos explorados: {nodos_explorados}")
    print(f"Número de nodos expandidos: {nodos_expandidos}")
    print(f"Coste de la solución: {coste}")
    print(f"Profundidad de la solución: {profundidad}")
else:
    print("No se encontró solución.")

TypeError: string indices must be integers